In [1]:
import openclsim.apps as apps
import openclsim.model as model
import openclsim.plugins as plugins

import shapely
import scipy.stats as st
import plotly.graph_objects as go

import numpy as np

In [31]:
class BaseModel(apps.SimulationEnvironment):
    
    def define_offshore_environment(self):
        pass

    def define_entities(self):

        self.port = apps.Site(
            env=self.env,
            name="PORT",
            geometry=shapely.geometry.Point(0, 0),
            capacity=4,
            level=4,
            nr_resources=1
        )

        self.owf = apps.Site(
            env=self.env,
            name="OWF",
            geometry=shapely.geometry.Point(1, 1),
            capacity=4,
            level=0,
            nr_resources=1
        )

        self.aeolus = apps.InstallationEquipment(
            env=self.env,
            name="AEOLUS",
            geometry=self.port.geometry,
            capacity=2,
            level=0,
            nr_resources=1,
            unloading_rate=1/3600,
            loading_rate=1/3600,
            compute_v=lambda x: 10
        )

        # Return a list of all the entities
        return [self.port, self.aeolus, self.owf]

    def define_operation(self):

        # Base activities
        load_component = model.ShiftAmountActivity(
            env=self.env,
            registry=self.registry,
            name="TRANSFER COMPONENT TO DECK.",
            processor=self.aeolus,
            origin=self.port,
            destination=self.aeolus,
            duration=3600,
            amount=1
        )

        transit_to_owf = model.MoveActivity(
            env=self.env,
            registry=self.registry,
            name="TRANSIT TO OFFSHORE SITE.",
            mover=self.aeolus,
            destination=self.owf,
            duration=3600
        )

        position = model.BasicActivity(
            env=self.env,
            registry=self.registry,
            name="(RE)POSITION ONTO LOCATION.",
            duration=3600
        )

        install_component = model.ShiftAmountActivity(
            env=self.env,
            registry=self.registry,
            name="INSTALL COMPONENT ON SITE.",
            processor=self.aeolus,
            origin=self.aeolus,
            destination=self.owf,
            duration=3600,
            amount=1
        )

        transit_to_port = model.MoveActivity(
            env=self.env,
            registry=self.registry,
            name="TRANSIT TO PORT.",
            mover=self.aeolus,
            destination=self.port,
            duration=3600 + 3600
        )

        
        # Loading cycle
        loading_cycle = model.WhileActivity(
            env=self.env,
            registry=self.registry,
            name="LOAD CYCLE.",
            sub_processes=[load_component],
            condition_event={"or": [
                {"type": "container", "concept": self.aeolus, "state": "full"},
                {"type": "container", "concept": self.port, "state": "empty"},
            ]}
        )

        # Unload/Installation cycle
        installation_cycle = model.WhileActivity(
            env=self.env,
            registry=self.registry,
            name="INSTALL CYCLE",
            sub_processes=[position, install_component],
            condition_event={"or": [
                {"type": "container", "concept": self.aeolus, "state": "empty"},
                {"type": "container", "concept": self.owf, "state": "full"},
            ]}
        )

        # Outer installation cycle
        outer_cycle = model.WhileActivity(
            env=self.env,
            registry=self.registry,
            name="OUTER CYCLE",
            sub_processes=[
                loading_cycle,
                transit_to_owf,
                installation_cycle,
                transit_to_port
            ],
            condition_event={"or": [
                {"type": "container", "concept": self.port, "state": "empty"},
                {"type": "container", "concept": self.owf, "state": "full"}
            ]}
        )

        # List base activities and store them as a property
        self.activities = [
            load_component, 
            transit_to_owf,
            position,
            install_component,
            transit_to_port
        ]
        
        # Return upstream activity
        return outer_cycle

In [32]:
x = BaseModel(start_date='0')
x.initiate_simulation()

'SUCCESSFUL.'

In [33]:
x.event_log.drop("ActivityID", axis=1)

,Timestamp,ActivityState,Description,PORT,AEOLUS,OWF
0,2020-01-01 00:00:00,START,TRANSFER COMPONENT TO DECK.,"{'container level': 4, 'geometry': POINT (0 0)}","{'geometry': POINT (0 0), 'container level': 0}","{'container level': 0, 'geometry': POINT (1 1)}"
1,2020-01-01 01:00:00,STOP,TRANSFER COMPONENT TO DECK.,"{'container level': 3, 'geometry': POINT (0 0)}","{'geometry': POINT (0 0), 'container level': 1}","{'container level': 0, 'geometry': POINT (1 1)}"
2,2020-01-01 01:00:00,START,TRANSFER COMPONENT TO DECK.,"{'container level': 3, 'geometry': POINT (0 0)}","{'geometry': POINT (0 0), 'container level': 1}","{'container level': 0, 'geometry': POINT (1 1)}"
3,2020-01-01 02:00:00,STOP,TRANSFER COMPONENT TO DECK.,"{'container level': 2, 'geometry': POINT (0 0)}","{'geometry': POINT (0 0), 'container level': 2}","{'container level': 0, 'geometry': POINT (1 1)}"
4,2020-01-01 02:00:00,START,TRANSIT TO OFFSHORE SITE.,"{'container level': 2, 'geometry': POINT (0 0)}","{'geometry': POINT (0 0), 'container level': 2}","{'container level': 0, 'geometry': POINT (1 1)}"
5,2020-01-01 03:00:00,STOP,TRANSIT TO OFFSHORE SITE.,"{'container level': 2, 'geometry': POINT (0 0)}","{'geometry': POINT (1 1), 'container level': 2}","{'container level': 0, 'geometry': POINT (1 1)}"
6,2020-01-01 03:00:00,START,(RE)POSITION ONTO LOCATION.,"{'container level': 2, 'geometry': POINT (0 0)}","{'geometry': POINT (1 1), 'container level': 2}","{'container level': 0, 'geometry': POINT (1 1)}"
7,2020-01-01 04:00:00,STOP,(RE)POSITION ONTO LOCATION.,"{'container level': 2, 'geometry': POINT (0 0)}","{'geometry': POINT (1 1), 'container level': 2}","{'container level': 0, 'geometry': POINT (1 1)}"
8,2020-01-01 04:00:00,START,INSTALL COMPONENT ON SITE.,"{'container level': 2, 'geometry': POINT (0 0)}","{'geometry': POINT (1 1), 'container level': 2}","{'container level': 0, 'geometry': POINT (1 1)}"
9,2020-01-01 05:00:00,STOP,INSTALL COMPONENT ON SITE.,"{'container level': 2, 'geometry': POINT (0 0)}","{'geometry': POINT (1 1), 'container level': 1}","{'container level': 1, 'geometry': POINT (1 1)}"
